In [2]:
import pandas as pd

movies = pd.read_csv('data/ratings_movies.csv')
movies.head()

display(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  100836 non-null  int64  
 1   userId      100836 non-null  int64  
 2   movieId     100836 non-null  int64  
 3   rating      100836 non-null  float64
 4   date        100836 non-null  object 
 5   title       100836 non-null  object 
 6   genres      100836 non-null  object 
dtypes: float64(1), int64(3), object(3)
memory usage: 5.4+ MB


None